In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/univer/Expociencia/Code/

/content/drive/MyDrive/univer/Expociencia/Code


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.7 MB/s 
     |████████████████████████████████| 596 kB 68.8 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import pandas as pd
import numpy as np
import gdown
from transformers import RobertaModel, RobertaTokenizerFast
from transformers import RobertaForMaskedLM
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch
from sklearn.metrics import f1_score
import time
from sklearn.metrics import accuracy_score
from pathlib import Path
from tqdm.auto import tqdm
from torch.nn import functional as F
import warnings
import logging
logging.basicConfig(level=logging.ERROR)
warnings.filterwarnings('ignore')

In [7]:
cd RoBERTa

/content/drive/MyDrive/univer/Expociencia/Code/RoBERTa


**Download dataset**

In [8]:
#train_data = pd.read_csv('train_yelp_clean.csv')
#val_data = pd.read_csv('val_yelp_clean.csv')
test_data = pd.read_csv('test_yelp_clean.csv')

In [9]:
#train_data['Score'] = train_data['Score'].apply(lambda x: x-1) 
#val_data['Score'] = val_data['Score'].apply(lambda x: x-1) 
test_data['Score'] = test_data['Score'].apply(lambda x: x-1) 

In [15]:
MAX_LEN = 128 
BATCH_SIZE = 32
NCLASSES= 5 
DropOut = 0.1
EPOCHS= 1
LEARNING_RATE=2e-5 
NAME_BERT_MODEL="bertin-project/bertin-roberta-base-spanish"
#NAME="bertin-roberta-base-spanish"
#myfile = Path(f'{NAME} LR {LEARNING_RATE}.txt')
#myfile.touch(exist_ok=True)
RoBERTa = RobertaModel.from_pretrained(NAME_BERT_MODEL, num_labels = 5, add_pooling_layer = False) 
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#RoBERTa.to(device)
print('ok')

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at bertin-project/bertin-roberta-base-spanish were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0
ok


In [10]:
tokenizer = RobertaTokenizerFast.from_pretrained("bertin-project/bertin-roberta-base-spanish")

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/831k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
tokenizer.convert_tokens_to_ids('<pad>')

1

In [11]:
class YELP(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len, modo):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode_truncation = modo
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item]) 
        label = self.labels[item]
        tokens = self.tokenizer.tokenize(review) 
        tokens = ['<s>'] + tokens + ['</s>']
        if len(tokens) < self.max_len:
            tokens = tokens + ['<pad>' for item in range(self.max_len-len(tokens))]
        elif len(tokens) > self.max_len:
            if self.mode_truncation == "head":
                tokens = tokens[:self.max_len-1] + ['</s>']    
            elif self.mode_truncation == "head+tail":
                tokens = tokens[:65] + tokens[-63:]#int((self.max_len)/2)
            elif self.mode_truncation == "tail":
                tokens = ['<s>'] + tokens[-int((self.max_len)/2):]
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        tokens_ids_tensor = torch.tensor(token_ids)
        attn_mask = (tokens_ids_tensor != 1).long()
        token_type_ids = torch.zeros_like(tokens_ids_tensor)
        
        return{
            'ids': tokens_ids_tensor,
            'mask': attn_mask,
            'targets': torch.tensor(self.labels[item], dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)}
 

def data_loader(df, tokenizer, max_len, batch_size, modo):
  dataset = YELP(
    reviews=df.Text.to_numpy(),
    labels = df.Score.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len,
    modo=modo
  )
  return DataLoader(dataset, batch_size= batch_size, num_workers=2)

In [14]:
#train_data_loader = data_loader(train_data, tokenizer, MAX_LEN, BATCH_SIZE, "head+tail")
#validation_data_loader = data_loader(val_data, tokenizer, MAX_LEN, BATCH_SIZE, "head+tail")
test_data_loader = data_loader(test_data, tokenizer, MAX_LEN, BATCH_SIZE, "head+tail")

In [16]:
class RoBERTaModel(torch.nn.Module):
    def __init__(self):
        super(RoBERTaModel, self).__init__()
        self.roberta = RoBERTa
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, NCLASSES)
        torch.nn.init.xavier_uniform_(self.classifier.weight)
    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.roberta(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [17]:
model = RoBERTaModel()
model.to(device)
print('ok')

ok


In [18]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_model (model,  epoch, data_loader, loss_fn, optimizer, device, n_examples):
  model=model.train()
  losses = []
  accuracy_global = []
  f1_score_global = []
  f1_weight_global = []
  min_loss = 0.9
  i = 0
  train_loop = tqdm(data_loader, leave=True, position = 0)
  for batch in train_loop:
    input_ids = batch['ids'].to(device)
    attention_mask = batch['mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
    labels = batch['targets'].to(device)
    outputs = model(input_ids, attention_mask, token_type_ids)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    f1_score_global.append(np.mean(f1_score(preds.cpu().detach().numpy(), labels.cpu().detach().numpy(), average=None)))
    f1_weight_global.append(np.mean(f1_score(preds.cpu().detach().numpy(), labels.cpu().detach().numpy(), average='weighted')))
    ac = accuracy_score(preds.cpu().detach().numpy(), labels.cpu().detach().numpy())
    accuracy_global.append(ac)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    #scheduler.step()
    optimizer.zero_grad()
    train_loop.set_description(f'epoch  {epoch}')
    train_loop.set_postfix(loss=np.mean(losses), acc = np.mean(accuracy_global), f1 = np.mean(f1_score_global), f1_weight = np.mean(f1_weight_global))

    if i%1000==0 and i>9999:
        validation_acc, validation_loss, validation_f1, history_val  = eval_model(
            model, epoch, validation_data_loader, loss_fn, device, len(val_data), 'Validación'
          )

        with open (f'{NAME} LR {LEARNING_RATE}.txt','a') as f:
                f.write(f'''    name: {NAME_BERT_MODEL}
                epoch: {i}
                Train:
                    -loss: {np.mean(losses)},
                    Accuracy: {np.mean(accuracy_global)} -- std: {np.std(accuracy_global)} -- var: {np.std(accuracy_global)**2},
                    f1_score: {np.mean(f1_score_global)} -- std: {np.std(f1_score_global)} -- var: {np.std(f1_score_global)**2},
                    f1_score: {np.mean(f1_weight_global)} -- std: {np.std(f1_weight_global)} -- var: {np.std(f1_weight_global)**2}
                Validation
                    -loss: {validation_loss},
                    Accuracy: {validation_acc} -- std: {np.std(history_val['accuracy'])} -- var: {np.std(history_val['accuracy'])**2},
                    f1_score: {validation_f1} -- std: {np.std(history_val['f1_score'])} -- var: {np.std(history_val['f1_score'])**2},
                    f1_score: {np.mean(history_val['f1_weighted'])} -- std: {np.std(history_val['f1_weighted'])} -- var: {np.std(history_val['f1_weighted'])**2}

                ---------------------------------

                ''')
        weight = {f'weight_model':model.state_dict()}
        if min_loss > validation_loss:
            torch.save(weight, f'Roberta_{i}_weightModelBERT.pth') 
            #min_loss = validation_loss
            
    i+=1
  return np.mean(accuracy_global), np.mean(losses), np.mean(f1_score_global), {'loss':losses, 
                                                                            'accuracy': accuracy_global,
                                                                            'f1_score': f1_score_global,
                                                                            'f1_weighted':f1_weight_global
                                                                            }


In [28]:
def eval_model(model, epoch, data_loader, loss_fn, device, n_examples, modo):
  model = model.eval()
  losses = []
  f1_weight_global = []
  accuracy_global = []
  f1_score_global = []
  val_loop = tqdm(data_loader, leave=True, position = 0)
  i=0
  with torch.no_grad():
    for batch in val_loop:
      input_ids = batch['ids'].to(device)
      attention_mask = batch['mask'].to(device)
      token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
      labels = batch['targets'].to(device)
      outputs = model(input_ids, attention_mask, token_type_ids)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      f1_score_global.append(np.mean(f1_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy(), average=None)))
      f1_weight_global.append(np.mean(f1_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy(), average='weighted')))
      ac = accuracy_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy())
      accuracy_global.append(ac)
      losses.append(loss.item())
      val_loop.set_description(f'epoch  {epoch}')
      val_loop.set_postfix(loss=np.mean(losses), acc = np.mean(accuracy_global), f1 = np.mean(f1_score_global), f1_weight = np.mean(f1_weight_global))
      i+=1
  val_loop.reset(0)
  return np.mean(accuracy_global), np.mean(losses), np.mean(f1_weight_global), {'loss':losses,
                                                                              'accuracy': accuracy_global,
                                                                              'f1_score': f1_score_global, 
                                                                              'f1_weighted':f1_weight_global
                                                                              }

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo tiene {count_parameters(model)} de parámetros')

El modelo tiene 124056581 de parámetros


In [ ]:
for epoch in range(EPOCHS):
  print('Epoch {} de {}'.format(epoch+1, EPOCHS))
  print('------------------')
  start_time = time.time() 
  train_acc, train_loss, train_f1, history_train = train_model(
    model, epoch, train_data_loader, loss_fn, optimizer, device, len(train_data)
  )
  finish_time = time.time()
  validation_acc, validation_loss, validation_f1, history_val = eval_model(
    model, epoch, validation_data_loader, loss_fn, device, len(val_data), 'Validación'
  )
  checkpoint = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict(),
             'history_by_epoch': history_train}
  torch.save(checkpoint, f'checkpoint_{epoch+1}.pth')
  print('Entrenamiento: Loss: {}, accuracy: {}, f1_score: {}'.format(train_loss, train_acc, train_f1))
  print('Validación: Loss: {}, accuracy: {}, f1_score: {}'.format(validation_loss, validation_acc, validation_f1))
  print('')
  elapsed_time = finish_time - start_time
  with open (f'{NAME} LR {LEARNING_RATE}.txt','a') as f:
    f.write(f'''    name: {NAME_BERT_MODEL}
    epoch: {epoch+1}
    learning rate: {LEARNING_RATE}
    bach size: {BATCH_SIZE}
    max len: {MAX_LEN}
    n clases: {NCLASSES}
    dropout: {DropOut}
    Train:
        -loss: {train_loss},
        Accuracy: {train_acc} -- std: {np.std(history_train['accuracy'])} -- var: {np.std(history_train['accuracy'])**2},
        f1_score: {train_f1} -- std: {np.std(history_train['f1_score'])} -- var: {np.std(history_train['f1_score'])**2},
        f1_weighted: {np.mean(history_train['f1_weighted'])} -- std: {np.std(history_train['f1_weighted'])} -- var: {np.std(history_train['f1_weighted'])**2}
    Validation
        -loss: {validation_loss},
        Accuracy: {validation_acc} -- std: {np.std(history_val['accuracy'])} -- var: {np.std(history_val['accuracy'])**2},
        f1_score: {validation_f1} -- std: {np.std(history_val['f1_score'])} -- var: {np.std(history_val['f1_score'])**2},
        f1_weighted: {np.mean(history_val['f1_weighted'])} -- std: {np.std(history_val['f1_weighted'])} -- var: {np.std(history_val['f1_weighted'])**2}
    time: {time.strftime("%H:%M:%S", time.gmtime(elapsed_time))}

    ---------------------------------

    ''')

Epoch 1 de 1
------------------


  0%|          | 0/19915 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (664 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (664 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: ignored

In [20]:
checkpoint = torch.load('Roberta_16000_weightModelBERT.pth')
checkpoint.keys() 

dict_keys(['weight_model'])

In [23]:
model.load_state_dict(checkpoint['weight_model'])

<All keys matched successfully>

In [21]:
epo = checkpoint['epoch']
weight = checkpoint['state_dict']
load_optimizer = checkpoint['optimizer']
model.load_state_dict(weight)
#optimizer.load_state_dict(load_optimizer)

KeyError: ignored

In [ ]:
for epoch in range(epo, EPOCHS):
  print('Epoch {} de {}'.format(epoch+1, EPOCHS))
  print('------------------')
  start_time = time.time() 
  train_acc, train_loss, train_f1, history_train = train_model(
    model, epoch, train_data_loader, loss_fn, optimizer, device, len(train_data)
  )
  finish_time = time.time()
  validation_acc, validation_loss, validation_f1, history_val = eval_model(
    model, epoch, validation_data_loader, loss_fn, device, len(val_data), 'Validación'
  )
  checkpoint = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict(),
             'history_by_epoch': history_train}
  torch.save(checkpoint, f'checkpoint_{epoch+1}.pth')
  print('Entrenamiento: Loss: {}, accuracy: {}, f1_score: {}'.format(train_loss, train_acc, train_f1))
  print('Validación: Loss: {}, accuracy: {}, f1_score: {}'.format(validation_loss, validation_acc, validation_f1))
  print('')
  elapsed_time = finish_time - start_time
  with open (f'/content/{NAME} LR {LEARNING_RATE}.txt','a') as f:
    f.write(f'''    name: {NAME_BERT_MODEL}
    epoch: {epoch+1}
    learning rate: {LEARNING_RATE}
    bach size: {BATCH_SIZE}
    max len: {MAX_LEN}
    n clases: {NCLASSES}
    dropout: {DropOut}
    warmup step: {WARMUP_STEP}
    Train:
        -loss: {train_loss},
        Accuracy: {train_acc} -- std: {np.std(history_train['accuracy'])} -- var: {np.std(history_train['accuracy'])**2},
        f1_score: {train_f1} -- std: {np.std(history_train['f1_score'])} -- var: {np.std(history_train['f1_score'])**2},
        f1_weighted: {np.mean(history_train['f1_weighted'])} -- std: {np.std(history_train['f1_weighted'])} -- var: {np.std(history_train['f1_weighted'])**2}
    Validation
        -loss: {validation_loss},
        Accuracy: {validation_acc} -- std: {np.std(history_val['accuracy'])} -- var: {np.std(history_val['accuracy'])**2},
        f1_score: {validation_f1} -- std: {np.std(history_val['f1_score'])} -- var: {np.std(history_val['f1_score'])**2},
        f1_weighted: {np.mean(history_val['f1_weighted'])} -- std: {np.std(history_val['f1_weighted'])} -- var: {np.std(history_val['f1_weighted'])**2}
    time: {time.strftime("%H:%M:%S", time.gmtime(elapsed_time))}

    ---------------------------------

    ''')

Epoch 2 de 4
------------------


  0%|          | 0/4478 [00:00<?, ?it/s]

In [29]:
print('------------------')            
validation_acc, validation_loss, validation_f1, history_val = eval_model(
        model, 0, test_data_loader, loss_fn, device, len(test_data), 'Test'
    )
print('Test: Loss: {}, accuracy: {}, f1: {}'.format(validation_loss, validation_acc, validation_f1))
print('')  

------------------


Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/767 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (596 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors


Test: Loss: 0.7526656463062281, accuracy: 0.6692878096479792, f1: 0.6702057948295356



In [27]:
print('Test: Loss: {}, accuracy: {}, f1: {}'.format(validation_loss, validation_acc, validation_f1))

Test: Loss: 0.6692878096479792, accuracy: 0.7526656463062281, f1: 0.6486134334912322
